In [1]:
import pandas as pd
import numpy as np
import json
import time
import itertools
import csv
import math
from collections import Counter

In [2]:
api_mat=[]
#time_mat=[]

def extraction(data):
    
    api=[]
    time=[]
    ds=json.dumps(data)
    behavior=data['behavior']
    processes=behavior['processes']
    for z in processes:
        calls=z['calls']
        for x in calls:
            api.append(x['api'])
            time.append(x['time'])
    time_list, api_list = zip(*[(x, y) for x, y in sorted(zip(time, api))])
    api_mat.append(api_list)
    #time_mat.append(time_list)

In [3]:
start=time.time()
for folder in range(1,101):
    file="C:/Users/sukul/Desktop/Ransomware Project/Benign/"+str(folder)+"/report.json"
    data = json.load(open(file))
    extraction(data)
N=len(api_mat)
exec_time=time.time()-start

In [20]:
arr=[]
tot=0
for i in range(N):
    tot+=len(api_mat[i])
    arr.append(len(api_mat[i]))
print("AVERAGE: ",tot/N)
print("MAX: ",max(arr))
print("MIN: ",min(arr))

AVERAGE:  153760.26
MAX:  3231951
MIN:  103


In [4]:
start=time.time()
n_grams=[[] for i in range(N)]

for i in range(N):
    arr=api_mat[i]
    for j in range(len(arr)-1):
        n_grams[i].append((arr[j],arr[j+1]))
        if(j<len(arr)-2):
            n_grams[i].append((arr[j],arr[j+1],arr[j+2]))
        if(j<len(arr)-3):
            n_grams[i].append((arr[j],arr[j+1],arr[j+2],arr[j+3]))
        if(j<len(arr)-4):
            n_grams[i].append((arr[j],arr[j+1],arr[j+2],arr[j+3],arr[j+4]))

exec_time=time.time()-start

In [1]:
with open('benign_ngrams_100samples.csv', 'w',newline="") as f:
    writer=csv.writer(f)
    writer.writerows(n_grams)

In [21]:
arr=[]
tot=0
for i in range(N):
    tot+=len(n_grams[i])
    arr.append(len(n_grams[i]))
print("AVERAGE: ",tot/N)
print("MAX: ",max(arr))
print("MIN: ",min(arr))

AVERAGE:  615031.04
MAX:  12927794
MIN:  402


In [5]:
start=time.time()
counter_ngrams=[]*N
for i in range(N):
    counter_ngrams.append(Counter(n_grams[i]))
exec_time=time.time()-start

In [6]:
D=[{} for i in range(N)]

In [7]:
start=time.time()
i=0

while(i<N):
    temp=list(set(n_grams[i]))
    j=0
    l=len(n_grams[i])
    t=len(temp)
    while(j<t):
        c=counter_ngrams[i]
        D[i].update({temp[j]: c[temp[j]]/l })
        j+=1
    i+=1
exec_time=time.time()-start

In [8]:
start=time.time()
d=[]
for i in range(N):
    temp=list(D[i].keys())
    for j in range(len(temp)):
        d.append(temp[j])
d=set(d)
d=list(d)
exec_time=time.time()-start 

In [9]:
len(d)

507750

In [9]:
idf={}

In [10]:

start=time.time()

i=0
l=len(d)
while(i<l):
    count=0
    for j in range(N):
        if d[i] in D[j]:
            count+=1
    idf[d[i]]=math.log(N/count)
    i+=1


exec_time=time.time()-start  

In [11]:
tf_idf={}
mult=[]

In [12]:
i=0
l=len(d)

start=time.time()
gram=list(idf.keys())
Const=math.log(N)
while(i<l):
    if(idf[gram[i]]==Const):
        for j in range(N):
            if gram[i] in D[j]:
                tf_idf[gram[i]]=idf[gram[i]]* D[j].get(gram[i])
    else:
        mult.append(gram[i])
        tf_idf[gram[i]]=[]
    i+=1

exec_time=time.time()-start  

In [13]:
start=time.time()
for i in range(len(mult)):
    for j in range(N):
        if mult[i] in D[j]:
            tf_idf[mult[i]].append(idf[mult[i]]*D[j].get(mult[i]))

exec_time=time.time()-start  

In [14]:
for i in range(len(mult)):
    tf_idf[mult[i]]=max( tf_idf[mult[i]])

In [15]:
sorted_tf_idf={k:v for k,v in sorted(tf_idf.items(),key=lambda item: item[1])}

In [22]:
sorted_list=list(sorted_tf_idf.keys())
sorted_list.reverse()

In [18]:
df=[[] for i in range(N)]

In [29]:
start=time.time()

for i in range(N):
    for j in range(len(sorted_list)):
        df[i].append(counter_ngrams[i].get(sorted_list[j]))
        
exec_time=time.time()-start

In [87]:
with open('ransom_dataset_100samples.csv', 'w',newline="") as f:
    writer=csv.writer(f)
    writer.writerow(sorted_list)
    writer.writerows(df)